# 0. Administrivia

# 1. Definitions
To start thinking concretely about **machine learning**, we need to have a definition for it. 

## 1A. Machine
What is the definition of the word **machine** in our context?  There are a few possibilties: Your Grandma’s sewing machine? 
- Tool that helps the task
- Implies automation

<img src="images/sewing_machine.jpg" style="width: 500px;">

A Pentium 2? A 3 to 1 pulley system? 

<img src="images/z_drag.jpg" style="width: 500px;">

These are all examples of machines (as understood in common English).  However, when you hear the phrase Machine Learning in the news, they’re talking about something different. Indeed, the phrase **machine** is a little bit misleading. For everything we do, it would (in principle) be possible to do the same calculations on pen and paper. Instead we’re looking for sets of instructions that ingest some sort of input and produce some sort of outputs, in such a way that once we’ve settled on those instructions, we don’t need to continue to be involved. This has a name that is frequently used in Computer Science: an algorithm. In fact, a better name for this course might actually be algorithmic learning.

## 1B. Learning
So this is a course that deals in algorithms, and the mathematics that underlie those algorithms. But perhaps a more vacuous concept comes from the second word: learning. What does it mean to learn? Wikipedia defines it thusly: *Learning is the process of acquiring new or modifying existing knowledge, behaviors, skills, values, or preferences.* Do you like that definition? I think it’s stupid: modifying your behavior or skill isn’t learning, because you might be going backwards. For example, this notebook might be making you even more confused than you were to begin with, and you might be thinking about modifying your behavior in the direction of taking Liberal Studies 147: Underwater Basket Weaving instead (not to demean anyone, I did part of my undergrad at the Evergreen State College). In this case, you wouldn’t be getting better at the task you set out to do. And so there I’ve implicitly added a certain value judgement to this definition: learning needs to involve improvement and it needs to involve a fairly well defined subject over which you would like to learn. Improvement at what? Let’s consider that question with a little example.

## 2. What to do with data?
Imagine that you work for UNICEF or some other multinational non-profit human rights organization, and you're trying to decide where to allocate funding.  The word has come in that the European Union has provided a sizeable grant for humanitarian aid to the nation of Venezuela.  In particular, this funding has been earmarked for projects related to reducing the infant mortality rate.  However, the problem is that no one even knows what the infant mortality rate for Venezuela is to begin with!  Of course, this isn't true, but for the sake of argument, let's imagine that this information has never been collected.  We need to make a prediction about this number; **How shall we proceed?  What is your strategy**

- Availability of healthcare
- Per Capita Income
- Presence of diseases
- Birthing practices
- Clean water availability

## 2A.  The world is not random.
Perhaps we can identify some proxy that might be related.  A reasonable *hypothesis* might be that per capita income is related to infant mortality rate.  Let's imagine that we have this information for Venezuela.  **Can we use that number to make a prediction?  What other information do we need in order to use it?**

Before we can utilize our hypothesized relationship between PCI and IMR, we'll need to do some machine learning in order to determine the nature of that relationship.  How might we best quantify said relationship?  By looking at the relationship between PCI and IMR elsewhere.  Fortunately, a useful dataset for this purpose does exist, and I downloaded it from http://www.stat.ufl.edu/~winner/datasets.html, which has a ton of great datasets.  

We can load these data using [pandas](https://pandas.pydata.org/), which we import here:

In [ ]:
import pandas as pd

The data is in a text format with spaces as delimiters.  Pandas has a nice parsing utility that we can use to read the file

In [ ]:
data = pd.read_csv('datasets/birthrate.dat',header=0,sep=r"\s{2,}",engine='python',index_col=0)
ven_data = data.loc['Venezuela']
data = data.drop('Venezuela')

Notice that this dataset does actually include Venezuela, but I've split it from the training set for instructional purposes.    

Let's have a look at what we just imported.  The Jupyter notebook has support for pandas data objects, which means they'll print nicely if we just say...

In [ ]:
data

PCI is in the second data column, while IMR is in the fourth.  However, tabular data is hard to interpret.  Let's plot instead, using the python package [matplotlib](https://matplotlib.org/).

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.figsize'] = [12,8]

plt.plot(data['PCI'],data['Infant Mortality'],'ro')
plt.xlabel('Per Capita Income')
plt.ylabel('Infant Mortality Rate')
plt.show()

That weird point out to the right looks kind of like an outlier, so let's drop it manually (American exceptionalism at work).

In [ ]:
data = data.drop('United States')

Also, from a computer's perspective, it's better to deal with small numbers rather than large ones (we'll see why later).  As such, we'd like to *normalize* these data so that both PCI and IMR scale from 0 to 1.  We'll save the amount that we scaled by, so we can always convert back

In [ ]:
pci_min = data['PCI'].min()
pci_range = data['PCI'].max() - pci_min
x = (data['PCI'] - pci_min)/pci_range

imr_min = data['Infant Mortality'].min()
imr_range = data['Infant Mortality'].max() - imr_min
y = (data['Infant Mortality'] - imr_min)/imr_range

We can plot these data again, but rescaled and without the outlier.

In [ ]:
plt.plot(x,y,'ro')
plt.xlabel('Relative Per Capita Income')
plt.ylabel('Relative Infant Mortality Rate')
plt.show()


## 2B. Hypothesizing a model

Clearly, there's some sort of relationship here that might be useful to us, for our stated goal, which is to make a prediction about the IMR in Venezuela.  Recall that we don't know that number, but *we do know its PCI*.  Let's plot that as a vertical line.  


In [ ]:
plt.plot(x,y,'ro')
plt.axvline((ven_data['PCI'] - pci_min)/pci_range)
plt.xlabel('Relative Per Capita Income')
plt.ylabel('Relative Infant Mortality Rate')
plt.show()

**Take a moment and determine your strategy.  How will you use this information?**

Make a list of potential options, but there's a common theme between all of these elements: we're going to construct a function of $x$, that yields a prediction about $y$.

$$
\underbrace{y}_{\text{prediction}} = \underbrace{F}_{\text{model}}(\underbrace{x}_{\text{feature}},\ldots)
$$
These things have important names, and we're going to be revisiting them time after time for the remainder of the semester.  $y$ is going to be called the *prediction*.  What is $y$ for this problem?  $x$ in the Machine Learning literature is typically called a *feature*.  In this context, there's only one, but in the future there will be many, sometimes millions (this is where the term big data comes from).  

## 2C. Incorporating Observations
The above definition is actually incomplete.  There's an additional element to every model that we also need to write:
$$
\underbrace{y}_{\text{prediction}} = \underbrace{F}_{\text{model}}(\underbrace{x}_{\text{feature}},\underbrace{w}_{\text{parameter}})
$$
What is a parameter?  Let's look at a very specific model:
$$
y = w_0 + w_1 x
$$
What do these $w$ values do?  They change the behavior of the model.  And **why would we want to change the behavior of the model?  What happens if we select some arbitrary values for $w_0$ and $w_1$ and use the resulting model to make a prediction?**

In [ ]:
import numpy as np

plt.plot(x,y,'ro')
x_ven = (ven_data['PCI'] - pci_min)/pci_range

w_0 = np.random.rand()*2 - 1
w_1 = np.random.rand()*2 - 1

plt.plot(x_ven,w_0 + w_1*x_ven,'bo')
plt.plot(x,w_0 + w_1*x,'b-')

plt.xlabel('Relative Per Capita Income')
plt.ylabel('Relative Infant Mortality Rate')
plt.show()

Good prediction?

**Prediction is not learning**.  We need to adjust our model in order to make better predictions, which *is* learning.  The parameters $w$ are knobs that we get to turn in order to make that adjustment, with the data that we have (in this case matched pairs of PCI and IMR).  **But how do we define good versus bad?  Get together with your neighbors and try to decide on some way to measure the model's predictive accuracy for the observations that exist**.  

## 2D. The classic example
What did you decide on?  The most common thing, and something you've probably all seen before is this:
$$
I(\mathbf{x},\underbrace{\hat{\mathbf{y}}}_{\text{Observation}},\mathbf{w}) = \sum_{i=1}^m (F(x_i,\mathbf{w}) - \hat{y}_i)^2
$$

So now we come to *the* fundamental operation in machine learning.  Stated concretely, for this problem it might be the following:  Find $w_0,w_1$ such that 
$$
\sum_{i=1}^m (w_0 + w_1 x_i - \hat{y}_i)^2
$$
is as small as possible.

# 3. The general problem
We can also state this more generally:  For a model 
$$\underbrace{\mathbf{y}}_{\text{prediction}} = \underbrace{F}_{\text{model}}(\underbrace{\mathbf{x}}_{\text{features}},\underbrace{\mathbf{w}}_{\text{parameters}}),$$
Minimize 
$$\underbrace{I}_{\text{Cost}}(\mathbf{y},\underbrace{\hat{\mathbf{y}}}_{\text{data}},\mathbf{w})$$
with resepct to $\mathbf{w}$.

## 3A. The complexity of ML
This is a very simple problem statement, but ML is a huge field growing daily with enormous workforce demand.  Where does the challenge and complexity come from.  Essentially, the richness of machine learning comes from trying to answer three fundamental questions:
- **Model Selection**: What is $F(\mathbf{x},\mathbf{w})$ and what is $I(\mathbf{y},\hat{\mathbf{y}},\mathbf{w})$.

- **Model Optimization**: How do we go about adjusting $\mathbf{w}$ in order to minimize $I(\mathbf{y},\hat{\mathbf{y}},\mathbf{w})$?

- **Model Validation**: How do know (and ensure) that our predictions are good?